In [ ]:
import pandas as pd
import networkx as nx
import plotly.graph_objects as go

# Load your dataset
df = pd.read_csv('/content/commentf.csv')

# Create a bipartite graph
B = nx.Graph()

# Add nodes for users and videos
user_nodes = df['User'].unique()
video_nodes = df['VideoID'].unique()

B.add_nodes_from(user_nodes, bipartite='users')
B.add_nodes_from(video_nodes, bipartite='videos')

# Add edges between users and videos they commented on
for _, row in df.iterrows():
    B.add_edge(row['User'], row['VideoID'])

# Extract subgraph of only connected components involving multiple videos
user_video_map = df.groupby('User')['VideoID'].apply(list)
valid_users = [user for user, videos in user_video_map.items() if len(set(videos)) > 1]
filtered_nodes = valid_users + list(df[df['User'].isin(valid_users)]['VideoID'].unique())
filtered_graph = B.subgraph(filtered_nodes)

# Generate node positions using spring layout
pos = nx.spring_layout(filtered_graph, seed=42)

# Map nodes to their positions for plotting
node_x = []
node_y = []
node_colors = []
user_group = {}

for node in filtered_graph.nodes():
    node_x.append(pos[node][0])
    node_y.append(pos[node][1])

    if node in user_nodes:
        # Assign a unique color to each user group based on common videos
        user_videos = df[df['User'] == node]['VideoID'].unique()
        group_color = hash(tuple(sorted(user_videos))) % 10  # Creating a unique color for the group
        user_group[node] = group_color
        node_colors.append(group_color)
    else:
        node_colors.append('lightblue')  # Video nodes

# Create the edges for the graph
edge_x = []
edge_y = []
for edge in filtered_graph.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_y.append(y0)
    edge_x.append(x1)
    edge_y.append(y1)

# Create the plot using plotly
fig = go.Figure()

# Add edges to the plot
fig.add_trace(go.Scatter(x=edge_x, y=edge_y, line=dict(width=0.5, color='gray'), hoverinfo='none', mode='lines'))

# Add nodes to the plot
fig.add_trace(go.Scatter(x=node_x, y=node_y, mode='markers', hoverinfo='text',
                         marker=dict(color=node_colors, size=10, line=dict(width=2)),
                         text=[node for node in filtered_graph.nodes()]))

# Create a custom legend for user groups and videos
fig.update_layout(
    title='Interactive Knowledge Graph of YouTube Comments',
    showlegend=True,
    legend=dict(
        x=1.05, y=1, traceorder='normal', orientation='v',
        xanchor='left', yanchor='top'
    ),
    annotations=[dict(x=0.85, y=0.9, xref='paper', yref='paper', text='Users (Colored)', showarrow=False),
                 dict(x=0.85, y=0.85, xref='paper', yref='paper', text='Videos (Lightblue)', showarrow=False)],
    height=800, width=800,
    plot_bgcolor='white'
)

fig.show()
